In [1]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import SETTING
import importlib
import tool_func
from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import multiagent as magent
import matplotlib.pyplot as plt

"""这里是非强化学习部分"""
# 初始化
num_vehicle = 8
num_order = 2
num_city = 4
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (10, 30))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}

# 设置s_0
for time in range(TIME):
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G ,speed)
    for order in Orders.values():
        Total_order[order.id] = order

# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 128         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 100      # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = STATE_DIM
)
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 1:
        # agent.load_model(load_path)
        env.time = 0

    for time in range(TIME):
    
        group = [[], []]

        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
            """
            if order.matched is False:
                order.virtual_departure = order.departure 
            """
        if time != 0 and episode != 0:
            next_vehicle_states = vectorization_vehicle(Vehicles)
            # 改了，不再是total_order
            next_order_states = vectorization_order(orders_unmatched)
            # 这里防止梯度爆炸缩小了reward
            agent.update(vehicle_states, order_states, grid_reward, next_vehicle_states, next_order_states, True ,action)
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = vectorization_vehicle(Vehicles)
            # 这里也改了
            order_states = vectorization_order(orders_unmatched)
            
            action = agent.take_action(vehicle_states, order_states)
        
            ACTIONS.append(action)            
            reward = env.test_step(orders_unmatched,action)
            # 一个循环代码让我达到最优
            # 屁股后面的代码是为了让我达到最优
            # 这里是为了让我达到最优
           
            change_time = 0
            COUNT = 999
            min_reward = -999999
            min_action = action
            while reward != 0 and COUNT > 0:
                action = agent.take_action(vehicle_states, order_states)
                reward = env.test_step(orders_unmatched,action)
                COUNT -= 1
                if reward > min_reward:
                    min_reward = reward
                    min_action = action
            if COUNT == 0:
                reward = env.test_step(orders_unmatched,min_action)
            
            """
            COUNT = 999
            min_reward = -999999
            min_action = action
            while reward != 0 and COUNT > 0:
                action = agent.take_action(vehicle_states, order_states)
                reward = env.test_step(orders_unmatched,action)
                COUNT -= 1
                if reward > min_reward:
                    min_reward = reward
                    min_action = action
            if COUNT == 0:
                reward = env.test_step(orders_unmatched,min_action)
            for env_order in env.orders.values():
                for order_unmatched in orders_unmatched.values():
                    if env_order.id == order_unmatched.id:
                        order_unmatched.virtual_departure = env_order.virtual_departure
                        change_time += 1
            print(change_time)
            city_update_base_drl(env.cities, orders_unmatched ,time)
            """
            

        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)

        if len(group[0]) != 0:
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            temp_Lower_Layer.get_decision()
            temp_Lower_Layer.constrain_1()
            temp_Lower_Layer.constrain_2()
            temp_Lower_Layer.constrain_3()
            temp_Lower_Layer.constrain_4()
            temp_Lower_Layer.constrain_5()
            temp_Lower_Layer.model.setParam('OutputFlag', 0)
            total_penalty = cancel_penalty * order_canceled
            temp_Lower_Layer.set_objective(matrix)
        
            temp_Lower_Layer.model.optimize()

            if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                # save_results(temp_Lower_Layer,time)
                # print("Objective value:", temp_Lower_Layer.model.objVal)
                objval = temp_Lower_Layer.model.objVal 
            else:
                temp_Lower_Layer.model.computeIIS()
                temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                # print(f"{time}次，No optimal solution found.")
                self_update(Vehicles, G)
                objval = basic_cost(Vehicles, orders_unmatched)
                
            
            _, var_order = temp_Lower_Layer.get_decision()
            update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
            vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            
        else:
            
            self_update(Vehicles, G)
            # print(f"{episode}轮，{time}次，{len(group[1])}辆车不在城市")
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = basic_cost(Vehicles, orders_unmatched)
            # 利润（如果有）减去新增的取消订单
            
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # 防止梯度爆炸
            grid_reward =  objval + reward
            grid_rewards.append(grid_reward)
            # print(grid_reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            # base_vehicle_class.append(copy.deepcopy(Vehicles))
            # base_order_class.append(copy.deepcopy(Total_order))
            
            base_city_node.append(copy.deepcopy(city_node))
            """
            if episode == 1:
                first_revnue.append(objval)
            """
        else:
            # if base_revenue[time] != objval:
            #    print("base_revenue",time, objval,base_revenue[time])
            # if first_revnue[time] != objval:
            #    print("first_revenue",time, first_revnue[time], objval)
            """ 
            if base_vehicle[time] != group[0]:
                print("vehicle is different", len(base_vehicle[time]), len(group[0]))
            if base_city_node[time] != env.cities:
                print(time, base_city_node[time],"\n", env.cities)
            """
            
        # print(f"{len(orders_unmatched)}订单未被匹配,{order_canceled}订单超时,总利润为{objval},强化学习利润为{reward}")
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        # torch.save(agent.state_dict(), 'model_checkpoint.pth')
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")
    # grid_rewards.append(0)
    # save_path = f"actor_critic_model{episode}.pth"
    # load_path = f"actor_critic_model{episode}.pth"
    # agent.save_model(save_path)
plt.plot(grid_rewards, label='Grid Reward Curve')
plt.xlabel('Iteration')
plt.ylabel('Grid Reward')
plt.title('Reward Curve')
plt.legend()
plt.grid(True)
plt.show()
# print(find_duplicates_with_positions(ACTIONS))

第0次训练
Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


未加强化学习利润为619304.0,0次未求解
第1次训练
执行时间: 2.7062771320343018 秒,0次未求解，当前强化学习值为618727.0,利润为618727.0
第2次训练
执行时间: 167.0326371192932 秒,91次未求解，当前强化学习值为184484.0,利润为290184.0
第3次训练
执行时间: 168.90956950187683 秒,0次未求解，当前强化学习值为618641.0,利润为618641.0
第4次训练
执行时间: 171.83974146842957 秒,0次未求解，当前强化学习值为618455.0,利润为618655.0
第5次训练
执行时间: 280.2519998550415 秒,111次未求解，当前强化学习值为78127.0,利润为215327.0
第6次训练
执行时间: 404.32373547554016 秒,124次未求解，当前强化学习值为19505.0,利润为169605.0
第7次训练
执行时间: 532.7008144855499 秒,130次未求解，当前强化学习值为-5123.0,利润为152577.0
第8次训练


KeyboardInterrupt: 

# 动态订单

In [1]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import SETTING
import importlib
import tool_func
from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import dynamic_multiagent as dm 
import matplotlib.pyplot as plt

"""这里是非强化学习部分"""
# 初始化
num_vehicle = 16
num_order = 5
num_city = 7
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (10, 30))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}

# 设置s_0
for time in range(TIME):
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G ,speed)
    for order in Orders.values():
        Total_order[order.id] = order

# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 128         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 100      # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = dm.DynamicMultiAgentAC(
    device = DEVICE,
    vehicle_state_dim =  STATE_DIM_VEHICLE,
    order_state_dim = STATE_DIM_ORDER, 
    num_cities = num_city, 
    hidden_dim = HIDDEN_DIM, 
    max_orders= num_order*TIME
)
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    agent.total_orders = Total_order
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 1:
        # agent.load_model(load_path)
        env.time = 0

    for time in range(TIME):
    
        group = [[], []]

        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
            """
            if order.matched is False:
                order.virtual_departure = order.departure 
            """
        if time != 0 and episode != 0:
            next_vehicle_states = vectorization_vehicle(Vehicles)
            # 改了，不再是total_order
            next_order_states = vectorization_order(Total_order)
            # 这里防止梯度爆炸缩小了reward
            agent.update(vehicle_states, order_states, next_vehicle_states, next_order_states, action, grid_reward , mask)
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = vectorization_vehicle(Vehicles)
            # 这里也改了
            order_states = vectorization_order(Total_order)
            # 这里也加了
            mask = agent.generate_mask()
            print(mask.shape)
            action = agent.take_action(vehicle_states, order_states)
            if action:
                print(action)
            ACTIONS.append(action)            
            reward = env.dynamic_step(Total_order,action,mask)
            # 一个循环代码让我达到最优
            # 屁股后面的代码是为了让我达到最优
            # 这里是为了让我达到最优
           
            change_time = 0
            COUNT = 999
            min_reward = -999999
            min_action = action
            while reward != 0 and COUNT > 0:
                action = agent.take_action(vehicle_states, order_states)
                reward = env.dynamic_step(Total_order,action,mask)
                COUNT -= 1
                if reward > min_reward:
                    min_reward = reward
                    min_action = action
            if COUNT == 0:
                reward = env.dynamic_step(Total_order,action,mask)
            
            """
            COUNT = 999
            min_reward = -999999
            min_action = action
            while reward != 0 and COUNT > 0:
                action = agent.take_action(vehicle_states, order_states)
                reward = env.test_step(orders_unmatched,action)
                COUNT -= 1
                if reward > min_reward:
                    min_reward = reward
                    min_action = action
            if COUNT == 0:
                reward = env.test_step(orders_unmatched,min_action)
            for env_order in env.orders.values():
                for order_unmatched in orders_unmatched.values():
                    if env_order.id == order_unmatched.id:
                        order_unmatched.virtual_departure = env_order.virtual_departure
                        change_time += 1
            print(change_time)
            city_update_base_drl(env.cities, orders_unmatched ,time)
            """
            

        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)

        if len(group[0]) != 0:
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            temp_Lower_Layer.get_decision()
            temp_Lower_Layer.constrain_1()
            temp_Lower_Layer.constrain_2()
            temp_Lower_Layer.constrain_3()
            temp_Lower_Layer.constrain_4()
            temp_Lower_Layer.constrain_5()
            temp_Lower_Layer.model.setParam('OutputFlag', 0)
            total_penalty = cancel_penalty * order_canceled
            temp_Lower_Layer.set_objective(matrix)
        
            temp_Lower_Layer.model.optimize()

            if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                # save_results(temp_Lower_Layer,time)
                # print("Objective value:", temp_Lower_Layer.model.objVal)
                objval = temp_Lower_Layer.model.objVal 
            else:
                temp_Lower_Layer.model.computeIIS()
                temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                # print(f"{time}次，No optimal solution found.")
                self_update(Vehicles, G)
                objval = basic_cost(Vehicles, orders_unmatched)
                
            
            _, var_order = temp_Lower_Layer.get_decision()
            update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
            vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            
        else:
            
            self_update(Vehicles, G)
            # print(f"{episode}轮，{time}次，{len(group[1])}辆车不在城市")
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = basic_cost(Vehicles, orders_unmatched)
            # 利润（如果有）减去新增的取消订单
            
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # 防止梯度爆炸
            grid_reward =  objval + reward
            grid_rewards.append(grid_reward)
            # print(grid_reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            # base_vehicle_class.append(copy.deepcopy(Vehicles))
            # base_order_class.append(copy.deepcopy(Total_order))
            
            base_city_node.append(copy.deepcopy(city_node))
            """
            if episode == 1:
                first_revnue.append(objval)
            """
        else:
            # if base_revenue[time] != objval:
            #    print("base_revenue",time, objval,base_revenue[time])
            # if first_revnue[time] != objval:
            #    print("first_revenue",time, first_revnue[time], objval)
            """ 
            if base_vehicle[time] != group[0]:
                print("vehicle is different", len(base_vehicle[time]), len(group[0]))
            if base_city_node[time] != env.cities:
                print(time, base_city_node[time],"\n", env.cities)
            """
            
        # print(f"{len(orders_unmatched)}订单未被匹配,{order_canceled}订单超时,总利润为{objval},强化学习利润为{reward}")
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        # torch.save(agent.state_dict(), 'model_checkpoint.pth')
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")
    # grid_rewards.append(0)
    # save_path = f"actor_critic_model{episode}.pth"
    # load_path = f"actor_critic_model{episode}.pth"
    # agent.save_model(save_path)
plt.plot(grid_rewards, label='Grid Reward Curve')
plt.xlabel('Iteration')
plt.ylabel('Grid Reward')
plt.title('Reward Curve')
plt.legend()
plt.grid(True)
plt.show()
# print(find_duplicates_with_positions(ACTIONS))

第0次训练
Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


未加强化学习利润为297320.0,139次未求解
第1次训练
(1, 720)


RuntimeError: Tensors must have same number of dimensions: got 1 and 2